In [8]:
import os
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import JSONLoader
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# OpenAI API 키를 환경 변수에서 로드
openai_api_key = os.getenv("OPENAI_API_KEY")


In [9]:
# JSON 파일 경로 설정
file_path = 'merged_data.json'

# JSONLoader의 jq_schema 설정 (필요한 필드만 포함)
jq_schema = """
.[] | {
    "filename": .filename,
    "date": .date,
    "conference_number": .conference_number,
    "question_number": .question_number,
    "meeting_name": .meeting_name,
    "generation_number": .generation_number,
    "committee_name": .committee_name,
    "meeting_number": .meeting_number,
    "session_number": .session_number,
    "agenda": .agenda,
    "law": .law,
    "qna_type": .qna_type,
    "context": .context,
    "context_summary": {
        "summary_q": .context_summary.summary_q,
        "summary_a": .context_summary.summary_a
    },
    "questioner": {
        "name": .questioner_name,
        "affiliation": .questioner_affiliation,
        "position": .questioner_position
    },
    "answerer": {
        "name": .answerer_name,
        "affiliation": .answerer_affiliation,
        "position": .answerer_position
    }
}
"""

# JSON 파일 로드
loader = JSONLoader(file_path, jq_schema=jq_schema, text_content=False)
documents = loader.load()

In [10]:
# 문서 분할 및 임베딩 생성
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

# CUDA 사용 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 임베딩 모델 설정 (GPU 사용)
embedding_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': device}
)

# FAISS 인덱스 설정
index_path = 'faiss_index'

# VectorStore 생성 또는 로드
if os.path.exists(index_path):
    print("저장된 FAISS 인덱스를 로드합니다...")
    vectorstore = FAISS.load_local(
        index_path,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True
    )
else:
    print("FAISS 인덱스를 생성합니다...")
    vectorstore = FAISS.from_documents(split_docs, embedding_model)
    vectorstore.save_local(index_path)


저장된 FAISS 인덱스를 로드합니다...


In [11]:
# Retriever 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

def json_search_tool(input_text):
    docs = retriever.invoke({"query": input_text})
    if docs:
        summaries = []
        for doc in docs:
            # Page Content나 Metadata가 딕셔너리인지 확인
            if isinstance(doc.page_content, dict):
                data = doc.page_content
            elif isinstance(doc.page_content, str):
                # 만약 page_content가 문자열이면 JSON으로 파싱
                try:
                    data = json.loads(doc.page_content)
                except json.JSONDecodeError:
                    return "페이지 내용이 JSON 형식이 아닙니다."
            else:
                # Page Content가 문자열이나 딕셔너리가 아니면 건너뜁니다.
                continue

            # 필요한 정보 추출
            conference_number = data.get('conference_number', 'N/A')
            meeting_name = data.get('meeting_name', 'N/A')
            generation_number = data.get('generation_number', 'N/A')
            committee_name = data.get('committee_name', 'N/A')
            agenda = data.get('agenda', 'N/A')
            law = data.get('law', 'N/A')
            date = data.get('date', 'N/A')
            questioner = data.get('questioner', {})
            answerer = data.get('answerer', {})
            context = data.get('context', 'N/A')

            # 질문자 정보
            questioner_name = questioner.get('name', 'N/A')
            questioner_affiliation = questioner.get('affiliation', 'N/A')
            questioner_position = questioner.get('position', 'N/A')

            # 답변자 정보
            answerer_name = answerer.get('name', 'N/A')
            answerer_affiliation = answerer.get('affiliation', 'N/A')
            answerer_position = answerer.get('position', 'N/A')

            # 컨텍스트 요약
            context_summary = data.get('context_summary', {})
            summary_q = context_summary.get('summary_q', 'N/A')
            summary_a = context_summary.get('summary_a', 'N/A')

            # 상세 답변 생성
            summary = (
                f"{questioner_name} 의원이 {date}에 열린 {committee_name}에서 "
                f"'{agenda}'에 대해 질문하였습니다. 이 회의는 {generation_number} "
                f"국회 {meeting_name} (회의 번호: {conference_number})로, "
                f"{answerer_name} {answerer_position}이(가) 답변하였습니다.\n\n"
                f"주요 내용:\n{context}\n\n"
                f"질문 요약:\n{summary_q}\n"
                f"답변 요약:\n{summary_a}\n"
            )
            summaries.append(summary)
        return '\n\n'.join(summaries)
    else:
        return "해당하는 정보를 찾을 수 없습니다."



In [12]:
# 도구 정의
json_search_tool = Tool(
    name="JSONSearch",
    func=json_search_tool,
    description=(
        "This tool is used to search for specific information about a person or issue from the JSON document. "
        "It provides summary information about fields such as conference number, meeting name, generation number, "
        "committee name, agenda, law, Q&A type, context, context summary, "
        "questioner (name, affiliation, position), and answerer (name, affiliation, position)."
    )
)

# 도구 목록 생성
tools = [json_search_tool]

In [13]:
# LLM 정의 (max_tokens 및 request_timeout 설정)
llm = ChatOpenAI(
    model_name="o1-mini",
    temperature=0,
    openai_api_key=openai_api_key,
    max_tokens=2048,
    request_timeout=120,
)

# 메모리 설정
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 에이전트 초기화
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

# 시스템 프롬프트 설정
agent.agent.llm_chain.prompt.messages[0].prompt.template = (
    "반드시 한국어로 답변해주세요. "
    "당신은 사용자가 국회 JSON 문서에서 인물이나 사안에 대한 정보를 찾을 수 있도록 도와주는 AI 어시스턴트입니다. "
    "사용자가 특정 인물이나 사안에 대해 질문하면, JSONSearch 도구를 사용하여 해당 인물이 언급된 회의 정보를 찾아야 합니다. "
    "검색된 정보를 바탕으로 상세하고 구체적인 답변을 제공하세요. 답변에는 회의 번호, 회의 이름, 세대 번호, 위원회 이름, 안건, 법률, 회의 날짜, 질문자 및 답변자의 이름과 소속, 회의에서 논의된 주요 내용 및 구체적인 대화 내용이 포함되어야 합니다. "
    "사용자가 이해하기 쉽게 문단으로 구성하여 답변하세요. "
    "JSON 문서에서 정보를 찾을 수 없는 경우, 정중하게 정보를 찾을 수 없음을 사용자에게 알려주세요. "
    "도구의 사용에 대해 언급하지 말고, 필요한 정보만 제공하세요."
)

# 에이전트와 대화하는 함수 정의
def chat_with_agent(user_input):
    response = agent.run(input=user_input)
    return response

In [14]:
# 예시 대화 실행
user_input = "김영삼 대통령이 언급된 회의에 대한 정보를 알려주세요."
response = chat_with_agent(user_input)
print("Assistant:", response)



> Entering new AgentExecutor chain...
```json
{
    "action": "JSONSearch",
    "action_input": "김영삼"
}
```

AttributeError: 'dict' object has no attribute 'replace'